In [1]:
# Mark B. & Katharine P.
# GGS416 Assignment 7 
# Final Project Code

# Image collection

In [ ]:
import folium
import geopandas as gpd
from geojson import Point, Feature, FeatureCollection, dump
from shapely.geometry import Point, Polygon
import pandas as pd

long= XXXX
lat= XXXX

centerpoint = [long,lat]

xmin= long - 0.014438
ymin= lat - 0.014438
xmax= long + 0.014438
ymax= lat + 0.014438


features = []
polygon = Polygon([[xmin,
		  ymin],
		  [xmin,
		  ymax],
		  [xmax,
		  ymax],
		  [xmin,
		  ymax],
		  [xmin,
		  ymin]])
features.append(Feature(geometry=polygon, properties={"Area": "COLONYNAME"}))
feature_collection = FeatureCollection(features)

with open('COLONYNAME.geojson', 'w') as f:
   dump(feature_collection, f)

In [ ]:
m = folium.Map([long, lat], zoom_start=10)
boundsdata = r'LOCATION.geojson'
folium.GeoJson(boundsdata).add_to(m)

In [ ]:
import sentinelsat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
api = SentinelAPI('XXXX', 'XXXX','https://scihub.copernicus.eu/dhus')

footprint = geojson_to_wkt(read_geojson(boundsdata))

In [ ]:
products = api.query(footprint,
                     date = ('XXXX', 'XXXX'),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0, 20),
                     limit=5)

In [ ]:
areas = api.to_geodataframe(products)
areas.plot(column='uuid', cmap=None, figsize = (10,10))

In [ ]:
ax = areas.plot(figsize = (20,20))
areas.apply(lambda x: ax.annotate(text=x['uuid'], xy=x.geometry.centroid.coords[0], ha='center'), axis=1);

In [ ]:
import matplotlib.pyplot as plt

gdf2 = gpd.read_file(boundsdata)
f, ax = plt.subplots(1)
areas.plot(ax=ax,column='uuid',cmap=None,)
gdf2.plot(ax=ax)
plt.show()

In [ ]:
# Image Download

In [ ]:
import folium
import geopandas as gpd
from geojson import Point, Feature, FeatureCollection, dump
from shapely.geometry import Point, Polygon
import pandas as pd
import sentinelsat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
api = SentinelAPI('XXXX', 'XXXX','https://scihub.copernicus.eu/dhus')

data = [LONG,LAT,'DATE','DATE','COLONY NAME']

def collecting(x):
    
        xmin= x[0] - 0.014438
        ymin= x[1] - 0.014438
        xmax= x[0] + 0.014438
        ymax= x[1] + 0.014438
        features = []
        polygon = Polygon([[xmin,
                  ymin],
                  [xmin,
                  ymax],
                  [xmax,
                  ymax],
                  [xmin,
                  ymax],
                  [xmin,
                  ymin]])
        
        features.append(Feature(geometry=polygon, properties={"Area": "Atlis"}))
        feature_collection = FeatureCollection(features)
        
        with open('{}.geojson'.format(x[4]), 'w') as f:
           dump(feature_collection, f)
        
        boundsdata = r'C:{}.geojson'.format(x[4])
        
        footprint = geojson_to_wkt(read_geojson(boundsdata))
        
        products = api.query(footprint,
                     date = (x[2],x[3]),
                     platformname = 'Sentinel-2',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (0, 20),
                     limit=5)
        
        api.download_all(products)

collecting(data)

In [ ]:
# View Image

In [ ]:
import rasterio
from matplotlib import pyplot
import numpy as np

image_filenameB02 = "XXXXB02_10m.jp2"
image_filenameB03 = "XXXXB03_10m.jp2"
image_filenameB08 = "XXXXB08_10m.jp2"

my_imageB02 = rasterio.open(image_filenameB02)
my_imageB03 = rasterio.open(image_filenameB03)
my_imageB08 = rasterio.open(image_filenameB08)

print(my_imageB02)
print(my_imageB03)
print(my_imageB08)

In [ ]:
blue = my_imageB02.read(1)
green = my_imageB03.read(1)
nir = my_imageB08.read(1)

def scale(band): # scale values for display purposes
    return band /10000

blue = scale(my_imageB02.read(1))
green = scale(my_imageB03.read(1))
nir = scale(my_imageB08.read(1))

In [ ]:
x = np.dstack((nir, green, blue))
pyplot.imshow(x)

In [ ]:
guano_reordered = np.moveaxis(x, [0, 1, 2], [1, 2, 0])
guano_reordered.shape

In [ ]:
with rasterio.open(
    'guano.tif',                           
    'w',                                            
    driver='GTiff',                                 
    height=guano_reordered.shape[1],                  
    width=guano_reordered.shape[2],                   
    count=guano_reordered.shape[0],                   
    dtype=guano_reordered.dtype,                      
    crs=my_imageB02.profile['crs'],                    
    transform=my_imageB02.profile['transform']         
    ) as my_raster_writer:
        my_raster_writer.write(guano_reordered)      

print('Finished writing guano_reordered')

In [ ]:
# Image Clipping

In [ ]:
# Import rasterio and file
import rasterio

image_file = "colony.tif"

my_image = rasterio.open(image_file)
my_image

In [ ]:
# Clarify the bounds of the image
xmin, ymin, xmax, ymax = my_image.bounds

xmin, ymin, xmax, ymax

In [ ]:
# Import rasterio.mask and implament it with a geojson
from rasterio.mask import mask
with rasterio.open(image_file) as img:
    clipped, transform = mask(img, my_geojson, crop=True)

# Determine the coordinates of the desired study area
xmin = XXXX
ymin = XXXX
xmax = XXXX
ymax = XXXX

# Create the geojson
my_geojson = [
    {
        "type": "Polygon",
        "coordinates": [  
          [
            [
              xmin,
              ymin
            ],
            [
              xmax,
              ymin
            ],
            [
              xmax,
              ymax
            ],
            [
              xmin,
              ymax
            ],
            [
              xmin,
              ymin
            ]
          ],
        ]
      }
    ]

meta = my_image.meta.copy()
meta.update(
    {
    
        "transform": transform,
        "height":clipped.shape[1],
        "width":clipped.shape[2]
    }
)

# Write out the new .tif
with rasterio.open('CLIPPEDCOLONYNAME.tif', 'w', **meta) as dst:
    dst.write(clipped)
    